In [8]:
import numpy as np

## Core algorithms: NNA and RNNA

In [33]:
def tsp_nna(points, start_node=0):
    """Nearest-neighbor search TSP approximate solver."""
    
    seq, dist = [start_node], 0.0
    cur_node = start_node
    points_remaining = set(range(len(points))) - set([start_node])
    
    while points_remaining:
        cand_nodes = list(points_remaining)
        
        cur_point = points[cur_node]
        cand_points = points[cand_nodes]
        
        # Calculate Euclidean distance between current point and all candidate points
        dists = np.sum((cand_points - cur_point) ** 2, axis=1)
        next_node = cand_nodes[np.argmin(dists)]
        
        seq.append(next_node)
        dist += np.sqrt(np.min(dists))
        points_remaining.remove(next_node)
        cur_node = next_node
        
    return seq, dist

In [30]:
def tsp_rnna(points):
    """Repetitive nearest-neighbor search TSP approximate solver."""
    
    return min((tsp_nna(points, start_node=node) for node in range(len(points))),
               key=lambda (seq, dist): dist)

## Data helpers

In [45]:
import os
import re

BASE_DIR = "~/scr/tmp/tsp"
def read_tsp_instance(name, base_path=BASE_DIR):
    base_path = os.path.expanduser(base_path)
    stmt_file = os.path.join(base_path, name + ".tsp")
    soln_file = os.path.join(base_path, name + ".opt.tour")
    
    if not os.path.exists(stmt_file) or not os.path.exists(soln_file):
        raise ValueError("statement file %r or soln file %r missing"
                         % (stmt_file, soln_file))
        
    with open(stmt_file, "r") as stmt_f:
        stmt_str = stmt_f.read()
    with open(soln_file, "r") as soln_f:
        soln_str = soln_f.read()
        
    if "EDGE_WEIGHT_TYPE : EUC_2D" not in stmt_str:
        print "%s has non-Euclidean distances; skipping" % name
        return None
    
    cities = np.array(re.findall(r"^\s*\d+\s+([^\s]+)\s+([^\s]+)$",
                                 stmt_str, re.M),
                      dtype=np.float32)
    
    soln_str = re.search(r"TOUR_SECTION\n(.+)-1", soln_str, re.S).group(1)
    soln = [int(x) - 1 for x in soln_str.strip().split()]
    
    return cities, soln

In [47]:
# Read all TSP data into memory.
tsp_files = os.listdir(os.path.expanduser(BASE_DIR))
names, inputs, solns = [], [], []

for filename in tsp_files:
    if filename.endswith(".opt.tour"):
        filename = filename[:filename.index(".opt.tour")]
        ret = read_tsp_instance(filename)
        if ret is None:
            continue
        
        points, soln = ret
        names.append(filename)
        inputs.append(points)
        solns.append(soln)
        
print len(inputs)

att48 has non-Euclidean distances; skipping
gr120 has non-Euclidean distances; skipping
ulysses22 has non-Euclidean distances; skipping
fri26 has non-Euclidean distances; skipping
gr24 has non-Euclidean distances; skipping
gr48 has non-Euclidean distances; skipping
gr666 has non-Euclidean distances; skipping
lin105 has non-Euclidean distances; skipping
gr96 has non-Euclidean distances; skipping
bayg29 has non-Euclidean distances; skipping
pa561 has non-Euclidean distances; skipping
bays29 has non-Euclidean distances; skipping
ch150 has non-Euclidean distances; skipping
gr202 has non-Euclidean distances; skipping
brg180 has non-Euclidean distances; skipping
ch130 has non-Euclidean distances; skipping
berlin52 has non-Euclidean distances; skipping
ulysses16 has non-Euclidean distances; skipping
14


## Comparing RNNA solutions + optimal solutions

In [34]:
def compute_distance(points, trajectory):
    dist = 0.0
    for node, next_node in zip(trajectory, trajectory[1:]):
        point, next_point = points[node], points[next_node]
        dist += np.sqrt(np.sum((point - next_point) ** 2))
    return dist

In [49]:
min_costs, heuristic_costs = [], []

for name, points, solution in zip(names, inputs, solns):
    print name
    heuristic_soln, heuristic_cost = tsp_rnna(points)
    min_cost = compute_distance(points, solution)
    
    min_costs.append(min_cost)
    heuristic_costs.append(heuristic_cost)
    print "\t%10f\t%10f" % (min_cost, heuristic_cost)

eil76
	540.002389	571.146490
pcb442
	50336.333870	57585.752045
eil101
	635.984980	713.492530
a280
	2568.881100	2986.623464
tsp225
	3830.500000	4519.910987
rd100
	7888.736187	9095.832102
kroA100
	20997.356293	23137.514534
eil51
	423.900550	468.037741
pr76
	106228.117493	123389.070679
st70
	670.053447	715.075203
pr1002
	256647.984879	304288.384590
kroC100
	20718.451546	22637.144608
kroD100
	20929.027868	23599.601492
pr2392
	377962.820173	455008.255384


In [50]:
min_costs = np.array(min_costs)
heuristic_costs = np.array(heuristic_costs)

In [53]:
rel_error = (heuristic_costs - min_costs) / min_costs * 100
print rel_error, rel_error.mean(), rel_error.var()

[  5.76740062  14.40195902  12.18700947  16.26164652  17.99793726
  15.30151201  10.19251286  10.41215724  16.15481248   6.71912911
  18.56254579   9.26079373  12.76014175  20.38439526] 13.3117109379 18.4273440481


In [56]:
print np.median([len(points) for points in inputs]), [len(points) for points in inputs]

100.0 [76, 442, 101, 280, 225, 100, 100, 51, 76, 70, 1002, 100, 100, 2392]
